<a href="https://colab.research.google.com/github/Gireesheruvaram/LLM-Beginer/blob/vector-database-kata3-branch/kata3ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [ ]:
pip install openai faiss-cpu numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 62.3 MB/s eta 0:00:00


In [ ]:

import openai

# Set your OpenAI API Key (replace with your actual API key)


In [ ]:
import openai
import numpy as np

# Set your OpenAI API Key (replace with your actual API key)
openai.api_key = "apikey"

# Function to generate embeddings for a document using the new OpenAI API
def generate_embeddings(documents):
    embeddings = []
    for doc in documents:
        # Use the new API to create embeddings
        response = openai.embeddings.create(
            model="text-embedding-ada-002",  # Specify the model
            input=doc  # The document text to embed
        )
        # Access the embedding correctly in the new API version
        embeddings.append(response.data[0].embedding)  # Correct access for embeddings
    return np.array(embeddings).astype('float32')  # Convert embeddings to numpy array

# Example corpus of documents
documents = [
    "The capital of France is Paris.",
    "Python is a programming language.",
    "The Eiffel Tower is in Paris."
]

# Generate embeddings for the documents
document_embeddings = generate_embeddings(documents)

# Display the generated embeddings
print(document_embeddings)



[[ 0.01860841 -0.00030375 -0.00583763 ...  0.00547452 -0.01221526
  -0.01181181]
 [ 0.0062781  -0.0094983   0.00577819 ...  0.00373959  0.00366168
  -0.03736995]
 [ 0.00538476 -0.02160198 -0.0085917  ...  0.0165162   0.00435879
  -0.0331331 ]]


In [ ]:
import faiss

# Create a FAISS index with L2 distance metric
index = faiss.IndexFlatL2(document_embeddings.shape[1])  # L2 distance metric
index.add(document_embeddings)  # Add the embeddings to the index

# Example query to search for similar documents
query = "Where is Paris located?"

# Generate embedding for the query
query_embedding = generate_embeddings([query])

# Perform the search on the FAISS index
k = 3  # Number of nearest neighbors to retrieve
D, I = index.search(query_embedding, k)

# Retrieve the top k most similar documents
similar_documents = [documents[i] for i in I[0]]

# Display the results
print("Query:", query)
print("Most similar documents:")
for doc in similar_documents:
    print(doc)


Query: Where is Paris located?
Most similar documents:
The capital of France is Paris.
The Eiffel Tower is in Paris.
Python is a programming language.
